In [14]:
from my_data_reader import get_data
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import networkx as nx
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [15]:
data=get_data('Amazon')
print(data.keys())
print([(i,type(v)) for i,v in data.items()])

dict_keys(['adj', 'num_features', 'num_nodes', 'features_nonzero', 'adj_norm', 'adj_label', 'features', 'labels'])
[('adj', <class 'scipy.sparse.lil.lil_matrix'>), ('num_features', <class 'int'>), ('num_nodes', <class 'int'>), ('features_nonzero', <class 'int'>), ('adj_norm', <class 'tuple'>), ('adj_label', <class 'tuple'>), ('features', <class 'tuple'>), ('labels', <class 'numpy.ndarray'>)]


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [25]:
Adj = np.zeros(data['adj_norm'][2])
i=0
for cord in data['adj_norm'][0]:
    Adj[cord[0]][cord[1]]=data['adj_norm'][1][i]
    i+=1
Adj=torch.Tensor(Adj)
Adj.require_grad=False
Adj

tensor([[0.2500, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.3333]])

In [37]:
X=np.zeros(shape=data['features'][2],dtype='float32')
i=0
for cord in data['features'][0]:
    X[cord[0]][cord[1]]=1
    i+=1
print('feature matrix')
X=torch.tensor(X,requires_grad=False)
X

feature matrix


tensor([[1., 1., 1.,  ..., 1., 0., 0.],
        [0., 1., 1.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 1., 1., 0.],
        ...,
        [0., 0., 1.,  ..., 1., 1., 0.],
        [0., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.]])

In [38]:
data['features']

(array([[   0,    0],
        [   0,    1],
        [   0,    2],
        ...,
        [1417,   17],
        [1417,   18],
        [1417,   19]], dtype=int32),
 array([2.06610e-02, 3.33333e-01, 1.90000e-05, ..., 9.00000e-01,
        6.25900e-03, 6.81818e-01]),
 (1418, 21))

In [120]:
class GCNConv(nn.Module):
    def __init__(self, A, in_channels, out_channels):
        super(GCNConv, self).__init__()
        self.A_hat = A
        self.W     = nn.Parameter(torch.rand(in_channels,out_channels))
    
    def forward(self, X):
        out = torch.mm(torch.mm(self.A_hat, X), self.W)
        return out

class NLGCN(nn.Module):
    def __init__(self, channels):
        super(NLGCN, self).__init__()
        self.channels=channels
        self.c=nn.Parameter(torch.rand(channels,1))
        self.conv1D=nn.Conv1d(1,1,3,padding=1)
        
    def forward(self, X):
        Av=X.matmul(self.c)
        indices_sorted=Av.argsort()
        indices_reorder=torch.sort(indices_sorted).values
        print(1,np.shape(X))
        X=X[indices_sorted]
        shape=np.shape(X)
        X=torch.reshape(X, (shape[0],shape[2]))
        X=torch.transpose(X,0,1)
        X=torch.reshape(X,(shape[2],shape[1],shape[0]))
        print(2,np.shape(X))
        X=self.conv1D(X)
        print(3,np.shape(X))
        X=torch.reshape(X,(shape[2],shape[0]))
        X=torch.transpose(X,1,0)
        print(3.5,np.shape(X))
        X=X[indices_reorder]
        X=torch.reshape(X,(shape[0],shape[2]))
        print(4,np.shape(X))
        return X

In [121]:
a=torch.Tensor([2,1,423,21,25,34,53,6,3,534])
x=a.argsort()
t=torch.sort(x).values

In [122]:
conv1 = GCNConv(A,21, 12)
conv2 = GCNConv(A,12, 10)
nlgcn=NLGCN(10)


In [123]:
H=conv1(X)
H=conv2(H)
H=nlgcn(H)

1 torch.Size([1418, 10])
2 torch.Size([10, 1, 1418])
3 torch.Size([10, 1, 1418])
3.5 torch.Size([1418, 10])
4 torch.Size([1418, 10])


In [91]:
np.shape(H)

torch.Size([1418, 10])